In [46]:
# Load packages
import pandas as pd 
import requests
import geojson
import geopandas as gpd
import json


Bereid een testroute voor

In [47]:
# Load pieterpad
route_gdf = gpd.read_file("25_sittard_strabeek(1).gpx", layer = "tracks")

route_gdf = route_gdf[["name", "geometry"]]

# Convert the GeoDataFrame to GeoJSON format
geojson_data = route_gdf.to_json()

# Optionally, print or save the GeoJSON data
print(geojson_data)

# Save GeoJSON data to a file
with open("route.json", "w") as f:
    f.write(geojson_data)


{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"name": "Sittard - Strabeek"}, "geometry": {"type": "MultiLineString", "coordinates": [[[5.870663523674011, 50.998068703338504], [5.871002990752459, 50.9980050008744], [5.871136011555791, 50.99800198338926], [5.872686998918653, 50.99712800234556], [5.872686998918653, 50.99712800234556], [5.87306896224618, 50.99684201180935], [5.873696012422442, 50.99607498385012], [5.87446597404778, 50.99526101723313], [5.87446597404778, 50.99526101723313], [5.87538898922503, 50.99417204037309], [5.876298006623983, 50.99297502078116], [5.876298006623983, 50.99297502078116], [5.876850038766861, 50.992040019482374], [5.877259997650981, 50.99154003895819], [5.877770036458969, 50.99096001125872], [5.878594983369112, 50.9901840146631], [5.878594983369112, 50.9901840146631], [5.880267005413771, 50.98861500620842], [5.881638033315539, 50.98810002207756], [5.881638033315539, 50.98810002207756], [5.882891966030002, 50.98763

In [48]:
with open("route.json", "r") as file:
    geojson_data = json.load(file)


In [49]:
import requests
import json
import math

# Haversine formula to calculate distance between two coordinates (in kilometers)
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the distance between two points on the Earth using the Haversine formula.
    """
    R = 6371  # Radius of the Earth in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Distance in km

def get_bounding_box(route_feature, buffer=0.02):
    """
    Calculate the bounding box around a route feature with a buffer.
    """
    min_lon, min_lat = float('inf'), float('inf')
    max_lon, max_lat = float('-inf'), float('-inf')

    # Loop through all coordinates in the route (MultiLineString)
    for line in route_feature['geometry']['coordinates']:
        for coord in line:
            lon, lat = coord
            # Update min/max coordinates
            min_lon = min(min_lon, lon)
            min_lat = min(min_lat, lat)
            max_lon = max(max_lon, lon)
            max_lat = max(max_lat, lat)

    # Add buffer to the bounding box
    min_lon -= buffer
    min_lat -= buffer
    max_lon += buffer
    max_lat += buffer
    
    return min_lon, min_lat, max_lon, max_lat

def fetch_pois(min_lon, min_lat, max_lon, max_lat, facility_type):
    """
    Fetch Points of Interest (POIs) from the Overpass API within a bounding box.
    """
    if facility_type == "supermarket":
        overpass_query = f'''
        [out:json];
        (node[shop="supermarket"]({min_lat},{min_lon},{max_lat},{max_lon});
        );
        out body;
        '''
    if facility_type == "horeca":
        overpass_query = f'''
            [out:json];
            (
            node[amenity="cafe"]({min_lat},{min_lon},{max_lat},{max_lon});
            node[amenity="restaurant"]({min_lat},{min_lon},{max_lat},{max_lon});
            );
            out body;
            '''

    if facility_type == "bus": 
        overpass_query = f'''
        [out:json];
        (node[highway="bus_stop"]({min_lat},{min_lon},{max_lat},{max_lon});
        );
        out body;
        '''
    
    # Make the API request
    overpass_url = 'https://overpass-api.de/api/interpreter'
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching POIs:", response.status_code)
        return None

def get_closest_pois(route_feature, pois_data):
    """
    For each node in the route, find the closest POI and store it in a dictionary.
    """
    unique_pois = {}

    for element in pois_data['elements']:
        poi_lon, poi_lat = element['lon'], element['lat']
        
        # For each coordinate (node) in the route, calculate the closest POI
        for line in route_feature['geometry']['coordinates']:
            for coord in line:
                node_lon, node_lat = coord
                
                # Calculate distance to the POI
                distance = haversine(poi_lon, poi_lat, node_lon, node_lat)

                # If this POI is closer than the current one (or not yet assigned), assign it
                if (node_lon, node_lat) not in unique_pois or unique_pois[(node_lon, node_lat)]['distance'] > distance:
                    unique_pois[(node_lon, node_lat)] = {
                        "poi_name": element.get('tags', {}).get('name', 'Unknown'),
                        "coordinates": [poi_lon, poi_lat],
                        "distance": distance,
                        "properties": {**element['tags'], "distance": distance}
                    }

    return unique_pois

def add_pois_to_geojson(geojson_data, unique_pois):
    """
    Add unique POIs to the GeoJSON data.
    """
    search_results = []
    for poi_info in unique_pois.values():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": poi_info["coordinates"]
            },
            "properties": poi_info["properties"]
        }
        search_results.append(feature)

    geojson_data["features"].extend(search_results)
    return geojson_data

def process_route(geojson_data, facility_type):
    """
    Process the route and add closest unique POIs to the GeoJSON data.
    """
    # Get the first route feature
    route_feature = geojson_data['features'][0]
    
    # Step 1: Calculate bounding box around the route
    min_lon, min_lat, max_lon, max_lat = get_bounding_box(route_feature)
    
    # Step 2: Fetch POIs within the bounding box
    pois_data = fetch_pois(min_lon, min_lat, max_lon, max_lat, facility_type)
    
    if pois_data:
        # Step 3: Get closest unique POIs for each node
        unique_pois = get_closest_pois(route_feature, pois_data)
        
        # Step 4: Add unique POIs to the GeoJSON data
        updated_geojson = add_pois_to_geojson(geojson_data, unique_pois)
        return updated_geojson
    else:
        print("No POIs found.")
        return geojson_data



In [50]:
# Process the route and get the updated GeoJSON data
updated_geojson = process_route(geojson_data, facility_type="horeca")

# Print the updated GeoJSON data
print(json.dumps(updated_geojson, indent=4))

with open('route_with_pois.json', 'w') as f:
    json.dump(updated_geojson, f)


{
    "type": "FeatureCollection",
    "features": [
        {
            "id": "0",
            "type": "Feature",
            "properties": {
                "name": "Sittard - Strabeek"
            },
            "geometry": {
                "type": "MultiLineString",
                "coordinates": [
                    [
                        [
                            5.870663523674011,
                            50.998068703338504
                        ],
                        [
                            5.871002990752459,
                            50.9980050008744
                        ],
                        [
                            5.871136011555791,
                            50.99800198338926
                        ],
                        [
                            5.872686998918653,
                            50.99712800234556
                        ],
                        [
                            5.872686998918653,
              

In [51]:
import folium

# Create a map centered around the average coordinates of your data
map_center = [50.8273797, 5.7261734]  # Adjust to your preferred central coordinates
mymap = folium.Map(location=map_center, zoom_start=12)

# Add the GeoJSON data to the map with a tooltip showing the name property
folium.GeoJson(
    geojson_data,
    tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['Name:'])
).add_to(mymap)

# Display the map (Jupyter notebooks will render it automatically)
mymap
